In [1]:
# Imports
import requests #For requesting html
import re #For regular expressions
import time
import pandas as pd
import random
import datetime
import logging
from openpyxl import load_workbook

In [2]:
# # For logging
# logger = logging.getLogger()
# fhandler = logging.FileHandler(filename='reviews.log', mode='a')
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# fhandler.setFormatter(formatter)
# logger.addHandler(fhandler)
# logger.setLevel(logging.DEBUG)

In [3]:
def get_status():
    """
    Reads the excel with the company names, links, modified links, how many options there were at search
    and how much has been scraped yet and returns those different lists
    """
    wb = load_workbook('status.xlsx')
    sheet = wb.worksheets[0]
    companies = sheet['A']
    number_options = sheet['B']
    links = sheet['C']
    pages = sheet['E']
    
    return companies, number_options, links, pages

In [4]:
def cells_to_list(cells):
    """
    Convert openpyxl format cells to strings
    """
    cells_list = []
    for cell in cells:
        cells_list.append(cell.value)
    return cells_list

In [5]:
# Get credentials
f = open('credentials.txt', 'r') #Opens the credentials.txt file for reading
credentials = f.readlines() #Returns a list of strings with cookie and user agent

In [6]:
# Setup
payload = ""
headers = {
    'cookie': credentials[0].replace('\n', ''),
    'authority': "www.glassdoor.com",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    'accept-language': "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    'cache-control': "max-age=0",
    'referer': "https://nl.glassdoor.be/",
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    'sec-ch-ua-mobile': "?0",
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': "document",
    'sec-fetch-mode': "navigate",
    'sec-fetch-site': "same-origin",
    'sec-fetch-user': "?1",
    'upgrade-insecure-requests': "1",
    'user-agent': credentials[1].replace('\n', '')
    }

In [7]:
# Get responses
def get_response(url):
    """
    returns glassdoor review query for a url converted to raw string
    """
    response = requests.request("GET", url, data=payload, headers=headers)
    raw_response = repr(response.text)
    return raw_response

In [8]:
#Extracts reviews out of response
def get_reviews(response):
    review_pattern = re.compile(r'{\"__typename\":\"EmployerReview\".+?\"translationMethod\":.+?}')
    matches = review_pattern.finditer(response)
    reviews = []
    for i, match in enumerate(matches):
        reviews.append(match.group(0))
    return reviews

In [9]:
def extract_from_review(review):
    """
    Expects a review build like a string dictionnary, extracts each feature out of a single review, returns a proper dictionary
    """
    review_id = re.compile(r'\"reviewId\":(.+?),\"reviewDateTime"').search(review).group(1)
    review_datetime = re.compile(r'\"reviewDateTime\":\"(.+?)\",\"ratingOverall\"').search(review).group(1)
    rating_overall = re.compile(r'\"ratingOverall\":(.+?),\"ratingCeo\"').search(review).group(1)
    rating_ceo = re.compile(r'\"ratingCeo\":(.+?),\"ratingBusinessOutlook\"').search(review).group(1)
    rating_businessoutlook = re.compile(r'\"ratingBusinessOutlook\":(.+?),\"ratingWorkLifeBalance\"').search(review).group(1)
    rating_worklife_balance = re.compile(r'\"ratingWorkLifeBalance\":(.+?),\"ratingCultureAndValues\"').search(review).group(1)
    rating_culture_values = re.compile(r'\"ratingCultureAndValues\":(.+?),\"ratingDiversityAndInclusion\"').search(review).group(1)
    rating_diversity_inclusion = re.compile(r'\"ratingDiversityAndInclusion\":(.+?),\"ratingSeniorLeadership\"').search(review).group(1)
    rating_senior_leadership = re.compile(r'\"ratingSeniorLeadership\":(.+?),\"ratingRecommendToFriend\"').search(review).group(1)
    rating_recommend_friend = re.compile(r'\"ratingRecommendToFriend\":(.+?),\"ratingCareerOpportunities\"').search(review).group(1)
    rating_career_opport = re.compile(r'\"ratingCareerOpportunities\":(.+?),\"ratingCompensationAndBenefits\"').search(review).group(1)
    rating_compensation_benefits = re.compile(r'\"ratingCompensationAndBenefits\":(.+?),\"employer\"').search(review).group(1)
    is_current_job = re.compile(r'\"isCurrentJob\":(.+?),\"lengthOfEmployment\"').search(review).group(1)
    length_employment = re.compile(r'\"lengthOfEmployment\":(.+?),\"employmentStatus\"').search(review).group(1)
    employment_status = re.compile(r'"employmentStatus":(.+?),"jobEndingYear"').search(review).group(1)
    job_ending_year = re.compile(r'"jobEndingYear":(.+?),"jobTitle"').search(review).group(1)
    pros = re.compile(r'\"pros\":\"(.+?)\",\"prosOriginal\":').search(review).group(1)
    cons = re.compile(r'\"cons\":\"(.+?)\",\"consOriginal\":').search(review).group(1)
    advice = re.compile(r'\"advice\":(.+?),\"adviceOriginal\":').search(review).group(1)
    count_helpful = re.compile(r'\"countHelpful\":(.+?),\"countNotHelpful\":').search(review).group(1)
    count_nothelpful = re.compile(r'\"countNotHelpful\":(.+?),\"employerResponses\"').search(review).group(1)
    language_id = re.compile(r'\"languageId\":(.+?),\"translationMethod\"').search(review).group(1)
    post_title = re.compile(r'\"summary\":(.+?),\"summaryOriginal\"').search(review).group(1)

    
    review_data = {'review_id': review_id,
               'post_title': post_title,
               'review_datetime': review_datetime,
               'review_overall': rating_overall,
               'rating_ceo': rating_ceo,
               'rating_businessoutlook': rating_businessoutlook,
               'rating_worklifebalance': rating_worklife_balance,
               'rating_culture_values': rating_culture_values,
               'rating_diversity_inclusion': rating_diversity_inclusion,
               'rating_senior_leadership': rating_senior_leadership,
               'rating_recommend_friend': rating_recommend_friend,
               'rating_career_opport': rating_career_opport,
               'rating_compensation_benefits': rating_compensation_benefits,
               'is_current_job': is_current_job,
               'length_employment': length_employment,
               'employment_status': employment_status,
               'job_ending_year': job_ending_year,
               'pros': pros,
               'cons': cons,
               'advice': advice,
               'count_helpful': count_helpful,
               'count_nothelpful': count_nothelpful,
               'language_id': language_id}
    
    return review_data

In [10]:
def datestr_to_str(date_str):
    """
    Convert a string with format 2022-06-13T05:07:07.813 to a datetime object
    """
    t_index = date_str.index('T')
    date = date_str[:t_index]
    date_time = datetime.datetime.strptime(date, '%Y-%m-%d')
    return date_time

In [11]:
def url_to_df(url, i, company, max_date='2017-01-01', s=5, page=1):
    """

    Parameters
    ----------
    url : str
        Base url with no filter except maybe location.
        ex: 'https://www.glassdoor.com/Reviews/Deloitte-Reviews-E2763.htm'
    max_date : string
        Defines how far the scraper will go in the past. 
        WARNING: This features only works properly if sorted by most recent
        ex: '2022-05-09'
    company : string
        Defines how the csv's will be named before saved
    s : int
        Defines how much pause between each request to the server. The lower the faster
        but also the more at risk of getting banned.
    page : str
        Defines at what page the scraper will start
    i : int
        In what position the company is in the excel
        Is needed to save the status of scraping
        

    Returns
    -------
    A list of dictionaries containing the information extracted from the reviews.

    """
    # Pattern to extract the reviews out of the javascript response
    review_pattern = re.compile(r'{\"__typename\":\"EmployerReview\".+?\"translationMethod\":.+?}')
    
    # Transform string date into datetime object for comparison
    max_date = datetime.datetime.strptime(max_date, '%Y-%m-%d')
    
    # Useful later to loop over pages
    htm_index = url.index('.htm')
    
    # Intializing list for storing reviews
    reviews_parsed = []
    
    # Extract from first page
    urlp = url[:htm_index] + '_P' + str(page) + url[htm_index:]  # Creates the url for the first page
    response = get_response(urlp)
    reviews = get_reviews(response)
    for review in reviews:
        parsed_review = extract_from_review(review)
        reviews_parsed.append(parsed_review) #Gets all the reviews in a list
    
    clean_company_name = company.replace('.', '').replace(' ', '').replace('/', '')
    
    
    # Define latest date
    if reviews: # true if list not empty
        latest_date = reviews_parsed[-1]['review_datetime'] # Takes the data of the last review of the page
        
        time.sleep(random.uniform(s-1, s+1))   
        # Extracts from following pages
        while datestr_to_str(latest_date) > max_date: # Evaluates true if we haven't passed the max date
            page +=1
            urlp = url[:htm_index] + '_P' + str(page) + url[htm_index:]  # Creates the url for the first page
            time.sleep(random.uniform(s-1, s+1))
            response = get_response(urlp)
            reviews = get_reviews(response)
            for review in reviews:
                parsed_review = extract_from_review(review)
                reviews_parsed.append(parsed_review) #Gets all the reviews in a list
                
            if reviews: 
                latest_date = reviews_parsed[-1]['review_datetime']
                if page % 50 == 0: # Saves the reviews each multiple of ten pages
                    #pd.DataFrame(reviews_parsed[:-2]).to_csv(f'../../data/{company}_{page}.csv')
                    pd.DataFrame(reviews_parsed).to_csv(f'reviews/{clean_company_name}_{page}.csv', index=False) #Save
                    sheet.cell(row=i+1, column=5).value=page
                    wb.save('status.xlsx')
                    reviews_parsed = [] #Delete
                    logging.info(f'{urlp} successfully saved')
                    time.sleep(random.uniform(80, 130)) #Long break
                else: 
                    continue
            else:
                break
    else: # If the list is empty, we initialize a variable at unimportant date just to avoid crash later on
        latest_date = '2000-01-01T05:07:07.813' 
        
    
        
    # To save what's left, duplicates likely from the last page scraped
    pd.DataFrame(reviews_parsed).to_csv(f'reviews/{clean_company_name}_{page}.csv', index=False)
    sheet.cell(row=i+1, column=5).value='done'
    wb.save('status.xlsx')
    logging.info(f'{urlp} successfully saved')

In [12]:
glassdoor_filter = '?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=PART_TIME&filter.employmentStatus=INTERN&filter.employmentStatus=REGULAR'
wb = load_workbook('status.xlsx', data_only=True)
sheet = wb.worksheets[0]
companies_cells, number_options_cells, links_cells, pages_cells = get_status()
companies = cells_to_list(companies_cells)
number_options = cells_to_list(number_options_cells)
links = cells_to_list(links_cells)
links_filtered = [link + glassdoor_filter if link != '/' else '/' for link in links]
pages = cells_to_list(pages_cells)

for i in range(len(companies)):
    if (links_filtered[i] != '_') and (links_filtered[i] != '/'):
        if pages[i] == '_':
            try:
                url_to_df(links_filtered[i], i=i, company=companies[i], s=6, page=1) #You can add max_date
            except:
                time.sleep(random.uniform(20, 30))
                url_to_df(links_filtered[i], i=i, company=companies[i], s=6, page=1) #You can add max_date
        elif pages[i] == 'done':
            continue
        else:
            try:
                url_to_df(links_filtered[i], i=i, company=companies[i], s=6, page=pages[i]) #You can add max_date
            except:
                time.sleep(random.uniform(20, 30))
                url_to_df(links_filtered[i], i=i, company=companies[i], s=6, page=pages[i]) #You can add max_date
    else:
        continue